In [2]:
import ee

# setting up earth_engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/xQHuoujz3MDkFQZLGTdotvtCYQp-ssXLJCB25izh8Apr4XjsjdO-tR0

Successfully saved authorization token.


In [8]:
power_plants = ee.FeatureCollection("WRI/GPPD/power_plants")

task = ee.batch.Export.table.toDrive(power_plants, "power_plants", fileFormat="CSV")
task.start()

In [9]:
task.status()

{'state': 'COMPLETED',
 'description': 'power_plants',
 'creation_timestamp_ms': 1583588466664,
 'update_timestamp_ms': 1583588477594,
 'start_timestamp_ms': 1583588472607,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://drive.google.com/'],
 'id': 'QKYHUASAX3EIPK3RVS4WD6RO',
 'name': 'projects/earthengine-legacy/operations/QKYHUASAX3EIPK3RVS4WD6RO'}

In [15]:
import folium

def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
        
folium.Map.add_ee_layer = add_ee_layer

basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}       
        
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

dem = ee.Image('USGS/SRTMGL1_003')
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

my_map.add_child(folium.LayerControl())

display(my_map)